# 16 Convolution Layer

In [2]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

    LABEL
        0 -> NORMAL (3,393)
        1 -> PNEUMOTHORAX (2,093)
        2 -> PNEUMONIA (4,402)

# hyper parameter

In [17]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

# label, test_label

In [4]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


# image, test_image

In [6]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

# image_dir = os.getcwd() + "/dir_images/181126/"
# test_image_dir = os.getcwd() + "/dir_images/181126_test/"

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

for i in range(len(image_list)) :
    image_list[i] = image_dir + image_list[i]
    image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

for j in range(len(test_image_list)) :
    test_image_list[j] = test_image_dir + test_image_list[j]
    test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

image_array = np.reshape(image_array, [-1, image_width*image_height])
test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


# feeding value

In [7]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

is_training = tf.placeholder(dtype=tf.bool)

# learning layers

In [8]:
def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.max_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

with tf.device('/gpu:0') :
    layer0_1 = tf.layers.conv2d(x, 8, [3,3], padding='SAME')
    layer0_2 = tf.layers.conv2d(layer0_1, 128, [3,3], padding='SAME')
    layer1 = learning_layer(input_data=layer0_2, size=256, dropout=0.7, is_training=is_training)
    
    layer1_1 = tf.layers.conv2d(layer1, 256, [3,3], padding='SAME')
    layer1_2 = tf.layers.conv2d(layer1_1, 128, [3,3], padding='SAME')
    layer2 = learning_layer(input_data=layer1_2, size=256, dropout=0.7, is_training=is_training)
    
    layer2_1 = tf.layers.conv2d(layer2, 512, [3,3], padding='SAME')
    layer2_2 = tf.layers.conv2d(layer2_1, 512, [3,3], padding='SAME')
    layer2_3 = tf.layers.conv2d(layer2_2, 256, [3,3], padding='SAME')
    layer3 = learning_layer(input_data=layer2_3, size=512, dropout=0.7, is_training=is_training)
    
    layer3_1 = tf.layers.conv2d(layer3, 1024, [3,3], padding='SAME')
    layer3_2 = tf.layers.conv2d(layer3_1, 512, [3,3], padding='SAME')
    layer3_3 = tf.layers.conv2d(layer3_2, 1024, [3,3], padding='SAME')
    layer3_4 = tf.layers.conv2d(layer3_3, 512, [3,3], padding='SAME')
    layer3_5 = tf.layers.conv2d(layer3_4, 256, [3,3], padding='SAME')
    layer4 = learning_layer(input_data=layer3_5, size=64, dropout=0.7, is_training=is_training)

W1008 17:27:12.952884 16404 deprecation.py:323] From <ipython-input-8-b65cb6eb6fe2>:9: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1008 17:27:12.955851 16404 deprecation.py:506] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1008 17:27:13.242113 16404 deprecation.py:323] From <ipython-input-8-b65cb6eb6fe2>:3: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W1008 17:27:13.353787 16404 deprecation.py:323] From <ipython-input-8-b65cb6eb6fe2>:4: 

In [9]:
with tf.device('/gpu:0') :
    fully_layer = tf.contrib.layers.flatten(layer4)
    fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu, name='fully_connected_layer')
    fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')
#     fully_layer1 = fully_layer(input_data=layer4, size=256, drop_out=0.5, activation=tf.nn.relu, is_training=is_training,
#                                name_dense="fully_layer1", name_dropout="fully_dropout1")

    fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu, name='fully_connected_layer2')
    fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')

W1008 17:27:16.140368 16404 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1008 17:27:16.141338 16404 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1008 17:27:16.319881 16404 deprecation.py:323] From <ipython-input-9-6582b8de4263>:3: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [10]:
with tf.device('/gpu:0') :
    logits = tf.layers.dense(fully_layer2, label_len, activation=None)
    
    with tf.name_scope('pred') : 
        pred = tf.nn.softmax(logits)
    with tf.name_scope('loss') :
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
#         loss = -tf.reduce_sum(y_ * tf.log(pred))
    with tf.name_scope('train') :
        train = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    label_output = tf.argmax(pred, 1)
    label_input = tf.argmax(y_, 1)

In [11]:
print(label_output)
print(label_input)

Tensor("ArgMax_2:0", shape=(?,), dtype=int64, device=/device:GPU:0)
Tensor("ArgMax_3:0", shape=(?,), dtype=int64, device=/device:GPU:0)


In [12]:
saver = tf.train.Saver()

tf.summary.scalar('scalar_loss', loss)

image_shaped_input = tf.reshape(x, [-1, image_resize, image_resize, 1])
tf.summary.image('input', image_shaped_input)

<tf.Tensor 'input:0' shape=() dtype=string>

In [18]:
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
#     f = open(log_dir+log_file, 'w')
    
#     saver = tf.train.import_meta_graph('meta_data_name')
#     saver.restore(sess, tf.train.latest_checkpoint('saved'))
    
    sess.run(tf.global_variables_initializer())
    
#     for var in tf.trainable_variables() :
#         tf.summary.histogram(var.name, var)

#     merged_summary = tf.summary.merge_all()  
#     train_writer = tf.summary.FileWriter(merge_dir, sess.graph)
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    
#     sensitivity_acc = 0
#     specificity_acc = 0
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_})    

#             _, _loss, _merge = sess.run([train, loss, merged_summary], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]],
#                                                                                   is_training: True, learning_rate: initial_rate*rate_})
            
#             saver.save(sess, save_dir, global_step=total_batch, write_meta_graph=False)
            
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
#     print("       SENSITIVITY ACC : ", '{:.5f}'.format(sensitivity_acc / 200))
#     print("       SPECIFICITY ACC : ", '{:.5f}'.format(specificity_acc / 100))
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
#     for cnt in range(len(correct_list)) :
#         data = correct_list[cnt]
#         f.write(str(data)+"\n")
#         print(data)
    
#     tf.train.write_graph(sess.graph, './', pb_, as_text=False)
#     tf.train.write_graph(sess.graph, './', txt_, as_text=True)
#     train_writer.add_graph(sess.graph)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
#     for cnt_incorrect in range(len(incorrect_list)) :
#         print(incorrect_list[cnt_incorrect])
    
#     f.close()
    
    coord.request_stop()
    coord.join(thread)

W1008 18:51:59.144603 16404 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  7.1564
Epoch :     2     Avg. cost =  2.8892
Epoch :     3     Avg. cost =  3.2954
Epoch :     4     Avg. cost =  1.8951
Epoch :     5     Avg. cost =  2.0799
Epoch :     6     Avg. cost =  1.6770
Epoch :     7     Avg. cost =  1.5491
Epoch :     8     Avg. cost =  1.7826
Epoch :     9     Avg. cost =  0.8288

000:07:53

Epoch :    10     Avg. cost =  1.0560
Epoch :    11     Avg. cost =  0.9398
Epoch :    12     Avg. cost =  1.1915
Epoch :    13     Avg. cost =  1.0425
Epoch :    14     Avg. cost =  0.8947
Epoch :    15     Avg. cost =  0.9240
Epoch :    16     Avg. cost =  0.9199
Epoch :    17     Avg. cost =  0.7539
Epoch :    18     Avg. cost =  0.7407
Epoch :    19     Avg. cost =  0.7532

000:16:44

Epoch :    20     Avg. cost =  0.7134
Epoch :    21     Avg. cost =  0.8508
Epoch :    22     Avg. cost =  0.8677
Epoch :    23     Avg. cost =  0.6154
Epoch :    24     Avg. cost =  0.5935
Epoch :    25     Avg. cost =  0.6495
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.0511
Epoch :   149     Avg. cost =  0.1139

002:12:30

Epoch :   150     Avg. cost =  0.1185
Epoch :   151     Avg. cost =  0.0525
Epoch :   152     Avg. cost =  0.0564
Epoch :   153     Avg. cost =  0.1819
Epoch :   154     Avg. cost =  0.0884
Epoch :   155     Avg. cost =  0.0672
Epoch :   156     Avg. cost =  0.0976
Epoch :   157     Avg. cost =  0.0747
Epoch :   158     Avg. cost =  0.0406
Epoch :   159     Avg. cost =  0.0692

002:21:25

Epoch :   160     Avg. cost =  0.0497
Epoch :   161     Avg. cost =  0.0455
Epoch :   162     Avg. cost =  0.1063
Epoch :   163     Avg. cost =  0.1759
Epoch :   164     Avg. cost =  0.1562
Epoch :   165     Avg. cost =  0.0437
Epoch :   166     Avg. cost =  0.0430
Epoch :   167     Avg. cost =  0.0473
Epoch :   168     Avg. cost =  0.0721
Epoch :   169     Avg. cost =  0.0413

002:30:19

Epoch :   170     Avg. cost =  0.0721
Epoch :   171     Avg. cost =  0.0780
Epoch :   172     Avg. cost =  0.0396
Epoch :   173 

Epoch :   290     Avg. cost =  0.0180
Epoch :   291     Avg. cost =  0.0221
Epoch :   292     Avg. cost =  0.0390
Epoch :   293     Avg. cost =  0.0120
Epoch :   294     Avg. cost =  0.0058
Epoch :   295     Avg. cost =  0.0581
Epoch :   296     Avg. cost =  0.0068
Epoch :   297     Avg. cost =  0.0129
Epoch :   298     Avg. cost =  0.0053
Epoch :   299     Avg. cost =  0.0050

004:26:06

Epoch :   300     Avg. cost =  0.0030
Epoch :   301     Avg. cost =  0.0036
Epoch :   302     Avg. cost =  0.0075
Epoch :   303     Avg. cost =  0.0029
Epoch :   304     Avg. cost =  0.0156
Epoch :   305     Avg. cost =  0.0168
Epoch :   306     Avg. cost =  0.0239
Epoch :   307     Avg. cost =  0.0038
Epoch :   308     Avg. cost =  0.0194
Epoch :   309     Avg. cost =  0.0014

004:35:00

Epoch :   310     Avg. cost =  0.0009
Epoch :   311     Avg. cost =  0.0007
Epoch :   312     Avg. cost =  0.0243
Epoch :   313     Avg. cost =  0.0012
Epoch :   314     Avg. cost =  0.0021
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.1120
Epoch :   438     Avg. cost =  0.0000
Epoch :   439     Avg. cost =  0.0002

006:30:40

Epoch :   440     Avg. cost =  0.0002
Epoch :   441     Avg. cost =  0.0001
Epoch :   442     Avg. cost =  0.0000
Epoch :   443     Avg. cost =  0.0001
Epoch :   444     Avg. cost =  0.0000
Epoch :   445     Avg. cost =  0.0000
Epoch :   446     Avg. cost =  0.0000
Epoch :   447     Avg. cost =  0.0000
Epoch :   448     Avg. cost =  0.0004
Epoch :   449     Avg. cost =  0.0000

006:39:34

Epoch :   450     Avg. cost =  0.0000
Epoch :   451     Avg. cost =  0.0000
Epoch :   452     Avg. cost =  0.0001
Epoch :   453     Avg. cost =  0.0151
Epoch :   454     Avg. cost =  0.0001
Epoch :   455     Avg. cost =  0.0001
Epoch :   456     Avg. cost =  0.0000
Epoch :   457     Avg. cost =  0.0796
Epoch :   458     Avg. cost =  0.0000
Epoch :   459     Avg. cost =  0.0000

006:48:28

Epoch :   460     Avg. cost =  0.0001
Epoch :   461     Avg. cost =  0.0000
Epoch :   462 

...

n:  242     label_input:  [0]     label_output:  [0]
n:  243     label_input:  [0]     label_output:  [0]
n:  244     label_input:  [0]     label_output:  [2]
n:  245     label_input:  [0]     label_output:  [1]
n:  246     label_input:  [0]     label_output:  [0]
n:  247     label_input:  [0]     label_output:  [1]
n:  248     label_input:  [0]     label_output:  [0]
n:  249     label_input:  [0]     label_output:  [1]
n:  250     label_input:  [0]     label_output:  [0]
n:  251     label_input:  [0]     label_output:  [0]
n:  252     label_input:  [0]     label_output:  [0]
n:  253     label_input:  [0]     label_output:  [1]
n:  254     label_input:  [0]     label_output:  [0]
n:  255     label_input:  [0]     label_output:  [0]
n:  256     label_input:  [0]     label_output:  [0]
n:  257     label_input:  [0]     label_output:  [1]
n:  258     label_input:  [0]     label_output:  [0]
n:  259     label_input:  [0]     label_output:  [0]
n:  260     label_input:  [0]     label_output

n:  402     label_input:  [1]     label_output:  [1]
n:  403     label_input:  [1]     label_output:  [1]
n:  404     label_input:  [1]     label_output:  [1]
n:  405     label_input:  [1]     label_output:  [1]
n:  406     label_input:  [1]     label_output:  [1]
n:  407     label_input:  [1]     label_output:  [1]
n:  408     label_input:  [1]     label_output:  [1]
n:  409     label_input:  [1]     label_output:  [1]
n:  410     label_input:  [1]     label_output:  [1]
n:  411     label_input:  [1]     label_output:  [0]
n:  412     label_input:  [1]     label_output:  [1]
n:  413     label_input:  [1]     label_output:  [0]
n:  414     label_input:  [1]     label_output:  [1]
n:  415     label_input:  [1]     label_output:  [1]
n:  416     label_input:  [1]     label_output:  [1]
n:  417     label_input:  [1]     label_output:  [1]
n:  418     label_input:  [1]     label_output:  [1]
n:  419     label_input:  [1]     label_output:  [1]
n:  420     label_input:  [1]     label_output

n:  558     label_input:  [1]     label_output:  [1]
n:  559     label_input:  [1]     label_output:  [1]
n:  560     label_input:  [1]     label_output:  [1]
n:  561     label_input:  [1]     label_output:  [1]
n:  562     label_input:  [1]     label_output:  [1]
n:  563     label_input:  [1]     label_output:  [1]
n:  564     label_input:  [1]     label_output:  [0]
n:  565     label_input:  [1]     label_output:  [1]
n:  566     label_input:  [1]     label_output:  [1]
n:  567     label_input:  [1]     label_output:  [1]
n:  568     label_input:  [2]     label_output:  [2]
n:  569     label_input:  [2]     label_output:  [2]
n:  570     label_input:  [2]     label_output:  [2]
n:  571     label_input:  [2]     label_output:  [2]
n:  572     label_input:  [2]     label_output:  [2]
n:  573     label_input:  [2]     label_output:  [2]
n:  574     label_input:  [2]     label_output:  [2]
n:  575     label_input:  [2]     label_output:  [2]
n:  576     label_input:  [2]     label_output

n:  719     label_input:  [2]     label_output:  [2]
n:  720     label_input:  [2]     label_output:  [2]
n:  721     label_input:  [2]     label_output:  [2]
n:  722     label_input:  [2]     label_output:  [2]
n:  723     label_input:  [2]     label_output:  [2]
n:  724     label_input:  [2]     label_output:  [2]
n:  725     label_input:  [2]     label_output:  [2]
n:  726     label_input:  [2]     label_output:  [2]
n:  727     label_input:  [2]     label_output:  [2]
n:  728     label_input:  [2]     label_output:  [0]
n:  729     label_input:  [2]     label_output:  [2]
n:  730     label_input:  [2]     label_output:  [2]
n:  731     label_input:  [2]     label_output:  [2]
n:  732     label_input:  [2]     label_output:  [2]
n:  733     label_input:  [2]     label_output:  [2]
n:  734     label_input:  [2]     label_output:  [2]
n:  735     label_input:  [2]     label_output:  [2]
n:  736     label_input:  [2]     label_output:  [2]
n:  737     label_input:  [2]     label_output

n:  876     label_input:  [2]     label_output:  [0]
n:  877     label_input:  [2]     label_output:  [1]
n:  878     label_input:  [2]     label_output:  [0]
n:  879     label_input:  [2]     label_output:  [2]
n:  880     label_input:  [2]     label_output:  [2]
n:  881     label_input:  [2]     label_output:  [2]
n:  882     label_input:  [2]     label_output:  [2]
n:  883     label_input:  [2]     label_output:  [2]
n:  884     label_input:  [2]     label_output:  [2]
n:  885     label_input:  [2]     label_output:  [2]
n:  886     label_input:  [2]     label_output:  [2]
n:  887     label_input:  [2]     label_output:  [2]
n:  888     label_input:  [2]     label_output:  [0]
n:  889     label_input:  [2]     label_output:  [2]
n:  890     label_input:  [2]     label_output:  [2]
n:  891     label_input:  [2]     label_output:  [2]
n:  892     label_input:  [2]     label_output:  [2]
n:  893     label_input:  [2]     label_output:  [2]
n:  894     label_input:  [2]     label_output